<a href="https://colab.research.google.com/github/XavierCachan/moduleIA_S4/blob/main/Reseau_Neurone_artificiel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Retour vers la partie 3 : Caractérisation d'une batterie par un neurone artificiel](https://colab.research.google.com/drive/1O0b8AnUaOWrjGoY7RDKRmHjWr2QeJZ_Z#scrollTo=PaKkvk0IJ916)

# **TP IA Partie 4 - Réseau de neurones artificiels pour la classification - IUT de Cachan GEII2 2024**
XM - Février 2024 - Version : 0.3

-----

Note : Pour avancer dans ce notebook, il suffit d'exécuter (petite flèche), ou compléter puis exécuter, les différents blocs de code placés ci-dessous.

Pour cette dernière partie du TP, nous allons attaquer un exercice plus complexe : traiter un problème de reconnaissance automatisée de chiffres écrits à la main, en utilisant la base de données MNIST ([lien Wikipedia](https://fr.wikipedia.org/wiki/Base_de_donn%C3%A9es_MNIST)). Cet exercice est (très) inspiré d'un exemple fourni par Google, que l'on peut retrouver [ici](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/multi-class_classification_with_MNIST.ipynb?utm_source=mlcc&utm_campaign=colab-external&utm_medium=referral&utm_content=multiclass_tf2-colab&hl=fr#scrollTo=wDlWLbfkJtvu).

**Rappel : Principe du codage d'une image**

Dans cette base de données, on trouve de nombreuses images d'apprentissage. Comme nous l'avons vu en S2, une image est codée en mémoire en donnant une valeur à chaque pixel, comme on peut le voir sur l'exemple ci-dessous d'une image 14x14 pixels :

![Principe de codage d'un caractère écrit à la main.](https://www.tensorflow.org/images/MNIST-Matrix.png)

Dans l'exercice qui va suivre, nos images ne font pas 14x14 mais 28x28 pixels (meilleure résolution). Chaque pixel est codé en niveau de gris par un entier 8 bits : 0 = blanc, 255 = noir. L'objectif de l'exercice sera donc d'arriver à entrainer un réseau de neurones capable de reconnaitre le chiffre entre 0 et 9 pour une image donnée.

**1. Import des modules nécessaires pour les calculs et l'affichage**

In [ ]:
import pandas as pd         # Module pour la construction d'un réseau de neurones
import tensorflow as tf     # TensorFlow est LE module utilisé pour l'entrainement d'un réseau de neurones
import numpy as np
import matplotlib.pyplot as plt

**2. Récupération des images d'entrainement et de test**

La base d'images fournie dispose cette fois d'un très grand nombre d'exemples. On va en profiter : afin de vérifier la qualité de l'entrainement du réseau que l'on va mettre en place, la base de caractéristiques initiales est divisée en 2 parties :  
- des images d'entrainement sur lequelles les poids/biais des neurones sont entrainés
- des images de test, non intégrées à l'entrainement (et donc que le réseau ne "connait" pas), sur lequelles on pourra vérifier que l'entrainement est bon (ou pas).

In [ ]:
# Récupération des caractéristiques (x) et des étiquettes (y) d'entrainement et de test
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()

Il est alors possible de "visualiser" une image caractéristique x de la base et de voir l'étiquette associée y :

In [ ]:
print("Il y a " + str(len(y_train)) + " images d'entrainement et " +  str(len(y_test)) + " images de test")

# Visualisation de l'image caractéristique #1713 utilisée pour l'entrainement
exemple = 1710    # Choix d'une caractéristique
print("Exemple de la caractéristique " + str(exemple) + " dont l'étiquette vaut " + str(y_train[exemple]))
x_train[exemple]

**3. Normalisation des caractéristiques**

Comme nous l'avons évoqué dans la partie 3, il est nécessaire d'effectuer un prétraitement des caractéristiques (x) avant de les fournir en entrée du réseau.

In [ ]:
# Normalisation des caractéristiques x pour la base d'entrainement et la base de test
x_train_norm = x_train / 255.0
x_test_norm = x_test / 255.0

Chaque pixel sera alors codé par un float entre 0 et 1.

In [ ]:
# Pixel 12 de la colonne 10
x_train_norm[exemple][10][12]

0.10588235294117647

**4. Création du réseau de neurones artificiels**

Comme dans l'exercice précédent, on commence par écrire la fonction définissant la topologie du réseau de neurones. Ce dernier est bien entendu plus complexe que celui à un seul neurone utilisé pour caractériser la batterie car :  
- Pour la batterie : on fournissait **une** entrée (un courant) et le réseau devait nous trouver **une** sortie (la tension U_bat associée)
- Pour la classification de chiffres : il a 28*28=784 entrées (les valeurs des pixels), et 10 sorties possibles (les chiffres de 0 à 9)

In [ ]:
# Fonction pour contruire un réseau de neurones, possiblement à plusieurs couches
def construction(taux_apprentissage):
  modele = tf.keras.models.Sequential()  # Définition d'un modèle de réseau séquentiel (ensemble de couches)

  # On commence par une couche pour transformer les caractéristiques (28x28) en un tableau 1D de taille 28x28 = 784(on "applatit" le tableau 2D).
  modele.add(tf.keras.layers.Flatten(input_shape=(28, 28)))

  # Ajout d'une première couche cachée de 4 neurones, fonction d'activation "Relu"
  modele.add(tf.keras.layers.Dense(units=4, activation='relu', input_shape=(784,)))    # Par defaut unit = 4

  # Ajout d'une couche de régularisation type "Dropout" : on désactive aléatoirement 20% des neurones
  modele.add(tf.keras.layers.Dropout(rate=0.2))

  # Ajout de la couche de sortie type "softmax", qui donne les probabilités associées à chaque étiquette possible
  # Les étiquettes étant pour nous les chiffres de 0 à 9, nous avons obligatoirement 10 possibilités en sortie
  modele.add(tf.keras.layers.Dense(units=10, activation='softmax'))

  # Construction du modèle à fournir à TensorFlow
  # La fonction d'erreur "crossentropy" est adaptée pour des problèmes dits de classification multi-classe comme le notre (étiquettes de 0 à 9)
  # L'optimiseur est Adam (c'était RMSprop dans l'exercice batterie)
  modele.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=taux_apprentissage),
                loss="sparse_categorical_crossentropy",
                metrics=['accuracy'])

  return modele

On définit ensuite la fonction d'entrainement utilisant TensorFlow.

In [ ]:
# Fonction pour entrainer le modèle de régression linéaire avec un petit nouveau : validation_split
def entrainement(modele, feature, label, epochs, batch_size, validation_split):
  # On fournit en arguments :
  # - modele : le réseau à entrainer
  # - feature : les "caractéristiques" d'entrée (x)
  # - label : les "étiquettes" de sortie (y)
  # - epochs : le nombre d'itérations (epochs)
  # - batch_size : le nombre de caractéristiques prises en compte dans une itération (entre 1 et le nb total disponible)
  # - validation_split (new!) : reserve une partie du lot caractéristiques/étiquettes d'entrainement pour évaluer le
  # réseau en cours d'apprentissage. Ex : validation_split = 0.2 => 80% pour l'entrainement, 20% de validation (!! à ne pas confondre avec nos images de test !!)

  history = modele.fit(x=feature,     # Fonction TensorFlow d'entrainement du réseau
                      y=label,
                      batch_size=batch_size,
                      epochs=epochs,
                      shuffle=True,
                      validation_split=validation_split)

  # Récupération de la liste des itérations
  epochs = history.epoch

  # Récupération des information à chaque itération
  hist = pd.DataFrame(history.history)

  return epochs, hist


On choisit les paramètres de l'entrainement et on construit la structure du réseau de neurones artificiels :

In [ ]:
taux_apprentissage=0.003    # Par defaut 0.003
epochs = 50                 # Par défaut 50
batch_size = 4000           # Par défaut 4000
validation_split = 0.2      # Par défaut 0.2

# Construction du réseau...
mon_modele = construction(taux_apprentissage)


Il est alors possible de vérifier si la structure est ok, et de voir le nombre de paramètres (et donc la taille!) du réseau que nous créons par la commande *summary* :

In [ ]:
mon_modele.summary()

Expliquer pourquoi la couche dense que nous avons créée contient 3140 paramètres... pour seulement 4 neurones !

**5. Entrainement du réseau de neurones artificiels**  
  
Tout semble ok, on peut passer à l'entrainement :

In [ ]:
# Entrainement
epochs, hist = entrainement(mon_modele, x_train_norm, y_train,
                            epochs, batch_size, validation_split)

# Affichage de l'évolution du taux de prédiction
fig,ax = plt.subplots(1,figsize=(8,5))
ax.plot (epochs,hist['accuracy'], linestyle="-", linewidth=1, label="Précision des prédictions")
ax.set_xlabel("Itérations")
ax.set_ylabel("Précision des prédictions")
ax.legend()
plt.show()

La précision semble déjà pas trop mal pour notre petit réseau à 4 neurones. Il reste à le tester...

**6. Evaluation du réseau de neurones artificiels entrainé**

**Evaluation globale**  
Le modèle obtenu peut alors être testé sur la partie de la base que nous avions initialement mise de côté pour la réserver aux tests. Elle est nommée (x_test, y_test) et n'a donc pas servi à l'apprentissage.

In [ ]:
# Analyse du modèle obtenu sur l'ensemble de la base de test pour obtenir la précision
print("Evaluation du modèle sur l'ensemble de la base de test (donc non connue par le réseau) :")
print("(Regarder la valeur de accuracy, si elle vaut 1 c'est parfait!) \n")
loss, acc = mon_modele.evaluate(x=x_test_norm, y=y_test, batch_size=batch_size)
print("Précision obtenue : {:5.2f}%".format(100 * acc))

**Evaluation d'un individu**  
Il est également possible d'obtenir la sortie du modèle pour une seule caractéristique x qui nous intéresse. Cette sortie contient donc pour chacune des 10 étiquettes la probabilité qu'elle soit "la bonne". Si tout se passe bien, la probabilité doit être maximale pour l'étiquette attendue.

In [ ]:
# Choisissons un exemple particulier de la base de test
exemple = 5    # Choix d'une caractéristique de la base de test (rappel : taille = 10000)  # Par défaut = 5
print("Exemple de la caractéristique " + str(exemple) + " de la base de test, dont l'étiquette à retrouver est " + str(y_test[exemple]))
x_test[exemple]

Quelles probabilités nous prédit le modèle pour cette caractéristique d'entrée ?

In [ ]:
# Prédiction sur un exemple particulier de la base de test
mon_modele.predict(x_test, verbose=0)[exemple]

Si la probabilité est maximale pour l'étiquette que l'on cherche, c'est que la prédiction est bonne ! Sinon, c'est une erreur du modèle:...  
Il ne reste donc plus qu'à trouver la probabilité maximale dans cette liste, et à afficher l'indice associé qui correspond directement à l'un des 10 chiffres possibles.

In [ ]:
# Recherche de l'indice de probabilité maximale
#np.max(mon_modele.predict(x_test, verbose=0)[exemple])      # Recherche du max
#np.argmax(mon_modele.predict(x_test, verbose=0)[exemple])   # Recherche de l'indice associé
print ("Notre modèle IA dit que le chiffre sur l'image est un : " + str(np.argmax(mon_modele.predict(x_test, verbose=0)[exemple])))

NameError: name 'np' is not defined

Testez pour une autre étiquette, par exemple l'étiquette 8 qui n'est vraiment pas simple à déchiffrer... même pour un humain ! (l'étiquette 18 est aussi assez difficile). Votre modèle propose-t-il une bonne prédiction dans ce cas ?

Peut on connaître tous les nombres pour lesquels il y a une mauvaise prédiction ? Oui, mais c'est un peu long... Testons déjà sur les 100 premières images de la base de test :

In [ ]:
# Récupération des probabilités (y) pour les 100 premières images de la base de test
y_pred = mon_modele.predict(x_test[0:100])
error = []

# Comparaison de la sortie à plus forte probabilité avec le chiffre attendu
for i in range (1,100):
  if np.argmax(mon_modele.predict(x_test, verbose=0)[i]) != y_test[i]:  # Si différence on ajoute à la liste
    error.append(i)
print(error)
print ("On a donc " + str(len(error)) + " erreur(s) de prédiction sur " + str(len(y_pred)) + " images de tests")

Pourrait-on améliorer en mettant 32 neurones au lieu de 4 dans la couche cachée ? (il faut donc repartir du point 4. Création du réseau de neurones artificiels)

Pourrait-on encore l'améliorer en mettant 256 neurones dans la première couche cachée et en ajoutant une seconde couche cachée de 128 neurones ? (on laissera une couche de dropout après chacune des 2 couches)

A partir de tout cela, on peut bien entendu mettre en place des systèmes de prédiction sur des ensembles plus complexes, comme ceux qui ont été étudiés lors des précédentes séances de TP du module IA. A vous de jouer :-)